<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/07_llmware_slim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from huggingface_hub import snapshot_download
# snapshot_download("llmware/slim-ner-tool", local_dir="slim_tools", local_dir_use_symlinks=False)
# snapshot_download("llmware/slim-topics-tool", local_dir="slim_tools", local_dir_use_symlinks=False)
# snapshot_download("llmware/slim-sentiment-tool", local_dir="slim_tools", local_dir_use_symlinks=False)

In [ ]:
# !pip install llmware

from llmware.models import ModelCatalog

# to load the model and make a basic inference
model = ModelCatalog().load_model("slim-sentiment-tool")

# model = ModelCatalog().load_model("slim-topics-tool")



## How to run SLIM models (i.e. Tools) - Approach 1

In [ ]:
earnings_transcripts = [
    "This is one of the best quarters we can remember for the industrial sector with significant growth across the "
    "board in new order volume, as well as price increases in excess of inflation.  We continue to see very strong "
    "demand, especially in Asia and Europe. Accordingly, we remain bullish on the tier 1 suppliers and would be "
    "accumulating more stock on any dips. ",

    "Not the worst results, but overall we view as negative signals on the direction of the economy, and the likely "
    "short-term trajectory for the telecom sector, and especially larger market leaders, including AT&T, Comcast, and"
    "Deutsche Telekom.",

    "This quarter was a disaster for Tesla, with falling order volume, increased costs and supply, and negative "
    "guidance for future growth forecasts in 2024 and beyond.",

    "On balance, this was an average result, with earnings in line with expectations and no big surprises to either "
    "the positive or the negative."
    ]

In [ ]:
for text in earnings_transcripts:
    # print(text)
    response = model.function_call(text)
    print(response['llm_response'])


{'sentiment': ['positive']}
{'sentiment': ['negative']}
{'sentiment': ['negative']}
{'sentiment': ['neutral']}


## How to run SLIM models (i.e. Tools) - Approach 2

In [ ]:
from llmware.agents import LLMfx

agent = LLMfx()
agent.load_tool("sentiment")

update: Launching LLMfx process
step - 	1 - 	creating object - ready to start processing.
step - 	2 - 	loading tool - sentiment


In [ ]:
# Change verbose to see more details
agent.verbose = False

text = "That is the worst thing that I have ever heard."
response = agent.exec_function_call("sentiment", text)

print("======================================================")
print("response:", response)

response: {'llm_response': {'sentiment': ['negative']}, 'dict_output': True, 'confidence_score': 0.866213860974206, 'choices': [('negative', 22198, 0.866213860974206), ('positive', 1066, 0.0672931902323938), ('neutral', 17821, 0.06547694723878902)]}


In [ ]:
# CODE FROM: https://github.com/llmware-ai/llmware/blob/main/examples/SLIM-Agents/slims-getting-started.py

""" Using SLIM tools as part of an agent workflow - introducing LLMfx class - this example shows how to:

    1.  Create an agent using the LLMfx class.
    2.  Load multiple specialized tools for the agent.
    3.  Execute a series of function-calls.
    4.  Generate a consolidated automatic dictionary report.

"""


from llmware.models import ModelCatalog
from llmware.agents import LLMfx


def create_multistep_report(customer_transcript):

    """ Creating a multi-step, multi-model agent workflow """

    #   create an agent using LLMfx class
    agent = LLMfx()
    # agent.verbose = False


    agent.load_work(customer_transcript)

    #   load tools individually
    # agent.load_tool("sentiment")
    # agent.load_tool("ner")

    #   load multiple tools
    agent.load_tool_list(["sentiment", "topics", "ner"])
    # agent.load_tool_list(["sentiment", "topics", "ner", "tags", "ratings", "answer"])


    #   start deploying tools and running various analytics

    #   first conduct three 'soft skills' initial assessment using 3 different models
    agent.sentiment()
    agent.topics()
    agent.ner()

    #   alternative way to execute a tool, passing the tool name as a string
    # agent.exec_function_call("sentiment")

    #   call multiple tools concurrently
    # agent.exec_multitool_function_call(["ner","topics","tags"])


    #   you can 'unload_tool' to release it from memory
    agent.unload_tool("ner")
    agent.unload_tool("topics")

    #   at end of processing, show the report that was automatically aggregated by key
    report = agent.show_report()

    #   displays a summary of the activity in the process
    activity_summary = agent.activity_summary()

    #   list of the responses gathered
    for i, entries in enumerate(agent.response_list):
        print("update: response analysis: ", i, entries)

    output = {"report": report, "activity_summary": activity_summary, "journal": agent.journal}

    return output



# sample customer transcript

customer_transcript = "My name is Michael Jones, and I am a long-time customer.  " \
                      "The Mixco product is not working currently, and it is having a negative impact " \
                      "on my business, as we can not deliver our products while it is down. " \
                      "This is the fourth time that I have called.  My account number is 93203, and " \
                      "my user name is mjones. Our company is based in Tampa, Florida."

output = create_multistep_report(customer_transcript)


update: Launching LLMfx process
step - 	1 - 	creating object - ready to start processing.
step - 	2 - 	loading new processing text - 1 new entries
step - 	3 - 	loading tool - sentiment
step - 	4 - 	loading tool - topics
step - 	5 - 	loading tool - ner
step - 	6 - 	executing function call - deploying - sentiment 
step - 	7 - 	executing function call - getting response - sentiment
				 -- llm_response - {'sentiment': ['negative']}
				 -- output type - dict
				 -- usage - {'input': 107, 'output': 8, 'total': 115, 'metric': 'tokens', 'processing_time': 0.826178789138794, 'type': 'dict'}
step - 	8 - 	analyzing response - sentiment
				 -- confidence score - 0.7877609640323416
				 -- analyzing response - sentiment': ['negative']}
				 -- analyzing response - [{'choice_1': ['negative', 0.7877609640323416, 22198], 'choice_2': ['neut', 0.1629893194280144, 17821], 'choice_3': ['pos', 0.048175048141058864, 1066]}]
				 -- analyzing response - [('negative', 22198, 0.7877609640323416), ('neutral

In [ ]:
output['report']

[{'sentiment': ['negative'],
  'topic': ['customer service'],
  'people': ['Michael Jones', 'mjones'],
  'place': ['Tampa', 'Florida'],
  'company': ['Mixco'],
  'misc': [],
  'source': {'text': 'My name is Michael Jones, and I am a long-time customer.  The Mixco product is not working currently, and it is having a negative impact on my business, as we can not deliver our products while it is down. This is the fourth time that I have called.  My account number is 93203, and my user name is mjones. Our company is based in Tampa, Florida.',
   'file_source': 'NA',
   'page_num': 'NA'}}]